In [2]:
import re, pandas as pd, numpy as np, requests, bs4, matplotlib.pyplot as plt
import wordcloud, nltk
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score, precision_score, accuracy_score
import wordcloud
import text_mining_utils as tm

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fuad9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Derive 3 matrices using 3 vectorisation techniques: counts, normalised counts and tfidf. Discuss the dimensionality and the differences between them

In [3]:
corpus = pd.read_csv('corpus.csv')
corpus

,Article,Class
0,\n \n NBA's Top 100 Players for ...,NBA
1,Rare ComicsTop 50 Marvel Comics Of 2015Top 50 ...,Marvel
2,LibraryLibraryComics and Graphic Novels\nColum...,DC
3,NBA Top 100 Players 2021-2022 | Ranking The Be...,NBA
4,Best DC Comics of 2019Comic Book HeraldA Comic...,DC
...,...,...
85,screenrant.comThe Best DC Comics Miniseries of...,DC
86,30 Best Marvel Comics (2022 UPDATED) Definitiv...,Marvel
87,Stephen Curry Ranks as NBA’s Most-Liked Player...,NBA
88,Best DC Comics storiesGamesRadar+ is supported...,DC


In [9]:
documents = list(corpus.Article)
baseline_count_matrix = tm.build_count_matrix(documents)
baseline_count_matrix

,NBA,'s,Top,100,Players,for,2019-20,:,Ranking,the,...,encompassed,blessed,2021.If,clicking,HERE,healthy.Fill,commenting,WordPress.com,Notify,Δ
0,3.0,3.0,4.0,3.0,1.0,1.0,3.0,2.0,1.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11.0,12.0,3.0,3.0,2.0,14.0,0.0,12.0,1.0,112.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,16.0,0.0,8.0,0.0,81.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.0,17.0,0.0,0.0,0.0,13.0,0.0,8.0,0.0,96.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,0.0,3.0,0.0,0.0,0.0,10.0,0.0,18.0,0.0,145.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88,0.0,53.0,0.0,0.0,0.0,13.0,0.0,13.0,0.0,179.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
baseline_tf_matrix = tm.build_tf_matrix(documents)
baseline_tf_matrix


,NBA,'s,Top,100,Players,for,2019-20,:,Ranking,the,...,encompassed,blessed,2021.If,clicking,HERE,healthy.Fill,commenting,WordPress.com,Notify,Δ
0,0.028037,0.028037,0.037383,0.028037,0.009346,0.009346,0.028037,0.018692,0.009346,0.102804,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.004098,0.000000,0.008197,0.000000,0.032787,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.004756,0.005188,0.001297,0.001297,0.000865,0.006053,0.000000,0.005188,0.000432,0.048422,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000402,0.000000,0.000000,0.000000,0.006431,0.000000,0.003215,0.000000,0.032556,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.000000,0.011281,0.000000,0.000000,0.000000,0.008626,0.000000,0.005309,0.000000,0.063703,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86,0.000000,0.000932,0.000000,0.000000,0.000000,0.003108,0.000000,0.005594,0.000000,0.045059,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
87,0.033784,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006757,0.000000,0.067568,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
88,0.000000,0.018976,0.000000,0.000000,0.000000,0.004654,0.000000,0.004654,0.000000,0.064089,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
baseline_tfidf_matrix = tm.build_tfidf_matrix(documents)
baseline_tfidf_matrix

,NBA,'s,Top,100,Players,for,2019-20,:,Ranking,the,...,encompassed,blessed,2021.If,clicking,HERE,healthy.Fill,commenting,WordPress.com,Notify,Δ
0,0.045809,0.022189,0.069084,0.069865,0.028340,0.001588,0.088876,0.004207,0.031046,0.010233,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000696,0.000000,0.001845,0.000000,0.003263,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.007770,0.004106,0.002397,0.003232,0.002622,0.001029,0.000000,0.001168,0.001436,0.004820,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000318,0.000000,0.000000,0.000000,0.001093,0.000000,0.000724,0.000000,0.003241,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0.000000,0.008928,0.000000,0.000000,0.000000,0.001466,0.000000,0.001195,0.000000,0.006341,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86,0.000000,0.000738,0.000000,0.000000,0.000000,0.000528,0.000000,0.001259,0.000000,0.004485,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
87,0.055198,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001521,0.000000,0.006725,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
88,0.000000,0.015018,0.000000,0.000000,0.000000,0.000791,0.000000,0.001048,0.000000,0.006379,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Choose at least 1 classification algorithm for baseline modelling;

In [15]:
dt_clf = DecisionTreeClassifier(random_state=1)
y = corpus.Class
y

0        NBA
1     Marvel
2         DC
3        NBA
4         DC
       ...  
85        DC
86    Marvel
87       NBA
88        DC
89    Marvel
Name: Class, Length: 90, dtype: object

### Apply the algorithm to the 3 matrices; document and discuss their performance using cross validation 

In [17]:
tm.crossvalidate_model(dt_clf, baseline_count_matrix, y, print_=True)

Accuracy: 0.90
Precision macro: 0.91
Recall macro: 0.90


(0.9, 0.9074603174603174, 0.9)

In [18]:
tm.crossvalidate_model(dt_clf, baseline_tf_matrix, y, print_=True)

Accuracy: 0.93
Precision macro: 0.94
Recall macro: 0.93


(0.9333333333333333, 0.9390476190476191, 0.9333333333333333)

In [19]:
tm.crossvalidate_model(dt_clf, baseline_tfidf_matrix, y, print_=True)

Accuracy: 0.93
Precision macro: 0.94
Recall macro: 0.93


(0.9333333333333333, 0.9390476190476191, 0.9333333333333333)